In [22]:
# test.py
import pandas as pd
from News_Crawler import crawling_news

df = pd.DataFrame(crawling_news.get_req())
df

,title,points,link
0,"Oracle, 이제 JavaScript를 놓아줘야 할 때입니다. (javascrip...",10,https://javascript.tm/
1,GN⁺: 책을 쓰지 말아야 하는 이유 (gwern.net),8,https://gwern.net/book-writing
2,The HTTP Query Method (ietf.org),6,https://www.ietf.org/archive/id/draft-ietf-htt...
3,GN⁺: MrBeast의 프로덕션 성공 비법 (유출된 PDF) (simonwilli...,32,https://simonwillison.net/2024/Sep/15/how-to-s...
4,SDR을 활용해서 pLTE 셀룰러 네트워크 구축하기 (youtube.com),3,https://www.youtube.com/watch?v=j2VBKvvmKKM
5,GN⁺: 인텔은 Sony Playstaion 6 계약을 어떻게 AMD에 뺏겼나 (...,6,https://www.reuters.com/technology/how-intel-l...
6,GN⁺: Swift 6 출시 (swift.org),4,https://www.swift.org/blog/announcing-swift-6/
7,GN⁺: 불타는 집에서 팀을 이끄는 방법 (peterszasz.com),14,https://peterszasz.com/how-to-lead-your-team-w...
8,OnlyFans의 놀라운 경제 구조 분석 (matthewball.co),14,https://www.matthewball.co/all/fansprofitandloss
9,GN⁺: Scrum이 당신을 스트레스 받게 하는 이유 (rethinkingsoftw...,9,https://rethinkingsoftware.substack.com/p/why-...


In [23]:
df['link']

0                                https://javascript.tm/
1                        https://gwern.net/book-writing
2     https://www.ietf.org/archive/id/draft-ietf-htt...
3     https://simonwillison.net/2024/Sep/15/how-to-s...
4           https://www.youtube.com/watch?v=j2VBKvvmKKM
5     https://www.reuters.com/technology/how-intel-l...
6        https://www.swift.org/blog/announcing-swift-6/
7     https://peterszasz.com/how-to-lead-your-team-w...
8      https://www.matthewball.co/all/fansprofitandloss
9     https://rethinkingsoftware.substack.com/p/why-...
10              https://github.com/PierreBeucher/novops
11                  https://news.hada.io/topic?id=16797
12                     https://lwn.net/Articles/990307/
13                                   https://konty.app/
14    https://www.cnbc.com/2024/09/16/amazon-jassy-t...
15                                https://openscad.org/
16    https://gizmodo.com/google-has-officially-kill...
17               https://www.gtf.io/musings/why-

In [24]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader(df['link'][5])
data = loader.load()

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0) 
all_splits = text_splitter.split_documents(data)

In [26]:
all_splits

[Document(metadata={'source': 'https://www.reuters.com/technology/how-intel-lost-sony-playstation-business-2024-09-16/', 'title': 'reuters.com', 'language': 'No language found.'}, page_content='reuters.comPlease enable JS and disable any ad blocker')]

In [27]:
import requests
from bs4 import BeautifulSoup

contents = []

for i in range(len(df['link'])):
    res = requests.get(df['link'][i], auth=("user", "pass")) 

    if res.status_code == 200:        
        soup = BeautifulSoup(res.text, 'lxml')
        # kill all script and style elements
        for script in soup(["script", "style"]):
            script.extract()  # rip it out

        # get text
        text = soup.get_text()

        # break into lines and remove leading and trailing space on each
        lines = (line.strip() for line in text.splitlines())
        # break multi-headlines into a line each
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        # drop blank lines
        text = '\n'.join(chunk for chunk in chunks if chunk)

        contents.append(text)
    elif res.status_code == 403:
        contents.append('e403')

    else:
        contents.append('e404')

In [28]:
pd.concat([df, pd.Series(contents, name='content')], axis=1)

,title,points,link,content
0,"Oracle, 이제 JavaScript를 놓아줘야 할 때입니다. (javascrip...",10,https://javascript.tm/,JavaScript™JavaScript™Deliver to: Oracle Corpo...
1,GN⁺: 책을 쓰지 말아야 하는 이유 (gwern.net),8,https://gwern.net/book-writing,Why To Not Write A Book · Gwern.net\nSkip to m...
2,The HTTP Query Method (ietf.org),6,https://www.ietf.org/archive/id/draft-ietf-htt...,The HTTP QUERY Method\nInternet-Draft\nThe HTT...
3,GN⁺: MrBeast의 프로덕션 성공 비법 (유출된 PDF) (simonwilli...,32,https://simonwillison.net/2024/Sep/15/how-to-s...,How to succeed in MrBeast production (leaked P...
4,SDR을 활용해서 pLTE 셀룰러 네트워크 구축하기 (youtube.com),3,https://www.youtube.com/watch?v=j2VBKvvmKKM,Broadcasting my own cellular - it works! | PLT...
5,GN⁺: 인텔은 Sony Playstaion 6 계약을 어떻게 AMD에 뺏겼나 (...,6,https://www.reuters.com/technology/how-intel-l...,e404
6,GN⁺: Swift 6 출시 (swift.org),4,https://www.swift.org/blog/announcing-swift-6/,Swift.org - Announcing Swift 6\nGet Started\nB...
7,GN⁺: 불타는 집에서 팀을 이끄는 방법 (peterszasz.com),14,https://peterszasz.com/how-to-lead-your-team-w...,How to Lead Your Team when the House Is on Fir...
8,OnlyFans의 놀라운 경제 구조 분석 (matthewball.co),14,https://www.matthewball.co/all/fansprofitandloss,Breaking Down OnlyFans’ Stunning Economics — M...
9,GN⁺: Scrum이 당신을 스트레스 받게 하는 이유 (rethinkingsoftw...,9,https://rethinkingsoftware.substack.com/p/why-...,Why Scrum is Stressing You Out - by Adam Ard\n...
